In [23]:
from google.colab import drive
mountpoint = "/content/drive"
drive.mount(mountpoint)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
pip install rouge-score pandas openpyxl


In [30]:
import pandas as pd
from rouge_score import rouge_scorer
import pandas as pd
from rouge_score import rouge_scorer
import numpy as np

In [24]:

df = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/summarization/billsum_v4_1/rouge_case_v2_legalBert.xlsx")

In [25]:
df

,Unnamed: 0,bill_id,text for summary,summarized_text,summary
0,0,s-1,"On April 8, 1993, BCH Energy, L.P. (¡°BCH¡± or...","On April 8, 1993, BCH Energy, L.P. entered int...",This legal case involves a dispute between a w...
1,1,s-2,Waterscape owns property located in Manhattan....,Waterscape entered into a Construction Managem...,"On June 28, 2007, Waterscape and Pavarini ente..."
2,2,s-3,NaN,NaN,NaN
3,3,s-4,NaN,NaN,NaN
4,4,s-5,Mandaree Public School District and Tooz Const...,Mandaree Public School District and Tooz Const...,Liberty Mutual was a surety that issued a perf...
5,5,s-6,"This case arises under the Miller Act, which r...","This case arises under the Miller Act, which r...",Defendant Shah contracted with the United Stat...
6,6,s-7,This litigation stems from contracts Defendant...,This litigation stems from contracts Defendant...,Defendant Dayton Board of Education(¡°Dayton¡±...
7,7,s-8,"On February 13, 2006, Beech Ridge Energy, LLC ...","Beech Ridge Energy, LLC and the Charleston Bui...","In 2006, Beech Ridge Energy and the Charleston..."
8,8,s-9,The Condominium consists of multiple buildings...,Lennar filed its amended complaint on April 29...,This case concerns multiple subcontracts for t...
9,9,s-10,"On September 17, 1904, the appellant Memphis T...","On September 17, 1904, the Memphis Trust Compa...","In 1904, Memphis Trust Company contracted with..."


In [35]:

# ✅ 2. "summarized_text" 및 "summary" 데이터 추출
if "summarized_text" in df.columns and "summary" in df.columns:
    summaries = df["summarized_text"].dropna().tolist()
    references = df["summary"].dropna().tolist()
else:
    raise ValueError("엑셀 파일에서 'summarized_text' 또는 'summary' 열을 찾을 수 없습니다.")

# ✅ 3. ROUGE 평가 함수 정의
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# ✅ 4. 전체 데이터셋에 대한 ROUGE 점수 계산
rouge1_p, rouge1_r, rouge1_f = [], [], []
rouge2_p, rouge2_r, rouge2_f = [], [], []
rougeL_p, rougeL_r, rougeL_f = [], [], []

for summary, reference in zip(summaries, references):
    scores = scorer.score(reference, summary)

    # ROUGE-1
    rouge1_p.append(scores["rouge1"].precision)
    rouge1_r.append(scores["rouge1"].recall)
    rouge1_f.append(scores["rouge1"].fmeasure)

    # ROUGE-2
    rouge2_p.append(scores["rouge2"].precision)
    rouge2_r.append(scores["rouge2"].recall)
    rouge2_f.append(scores["rouge2"].fmeasure)

    # ROUGE-L
    rougeL_p.append(scores["rougeL"].precision)
    rougeL_r.append(scores["rougeL"].recall)
    rougeL_f.append(scores["rougeL"].fmeasure)

# ✅ 5. 평균 ROUGE 점수 계산
rouge_avg = {
    "ROUGE-1 Precision": np.mean(rouge1_p),
    "ROUGE-1 Recall": np.mean(rouge1_r),
    "ROUGE-1 F1-score": np.mean(rouge1_f),
    "ROUGE-2 Precision": np.mean(rouge2_p),
    "ROUGE-2 Recall": np.mean(rouge2_r),
    "ROUGE-2 F1-score": np.mean(rouge2_f),
    "ROUGE-L Precision": np.mean(rougeL_p),
    "ROUGE-L Recall": np.mean(rougeL_r),
    "ROUGE-L F1-score": np.mean(rougeL_f),
}

# ✅ 6. 결과를 데이터프레임으로 변환 후 저장
rouge_df = pd.DataFrame([rouge_avg])
print(rouge_df)

   ROUGE-1 Precision  ROUGE-1 Recall  ROUGE-1 F1-score  ROUGE-2 Precision  \
0           0.413159        0.731373          0.521117           0.213952   

   ROUGE-2 Recall  ROUGE-2 F1-score  ROUGE-L Precision  ROUGE-L Recall  \
0         0.37546          0.269095           0.248795        0.439789   

   ROUGE-L F1-score  
0          0.313771  
